In [1]:
import numpy as np

import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv
from collections import defaultdict
import itertools

/Users/ghost/src/virtual_env/jupyther3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/ghost/src/virtual_env/jupyther3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
resident_df = pd.DataFrame()
resident_df = pd.read_csv( 'file/parsed_output/resident.csv') 
#hc_df.reset_index(level=0, inplace=True)
resident_df

,_id,_index_map,health_information.alcohol_within_week,health_information.times_exposed_to_t_smoke,personal_info.source_of_income,identity_consent.personal_id_no,identity_consent.town,dental_health.owning_toothbrush,identity_consent.identity_document,health_information.how_taking_prescribed_dose,...,personal_info.gender,pregnancy.is_first_pregnancy,health_information.has_health_insurance_plan,household.mostly_used_to_wash_hands,household.type_of_sanitation,household.no_of_people_household,dental_health.times_brush_teeth_daily,health_information.blood_pressure_1st_systole,end,health_information.diagnosed
0,173,NaN,NaN,NaN,NaN,NaN,_,NaN,BC,NaN,...,F,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2018-10-02T10:08:13.645+07,NaN
1,176,NaN,NaN,NaN,NaN,NaN,_,NaN,BC,NaN,...,M,NaN,NaN,NaN,NaN,4.0,NaN,NaN,2018-10-02T11:00:08.474+07,NaN
2,178,NaN,NaN,NaN,NaN,l,_,NaN,BC,NaN,...,F,NaN,NaN,NaN,NaN,5.0,NaN,NaN,2018-10-02T10:57:14.485+07,NaN
3,181,NaN,NaN,NaN,NaN,NaN,_,NaN,BC,NaN,...,F,NaN,NaN,NaN,NaN,5.0,NaN,NaN,2018-10-02T11:33:57.637+07,NaN
4,179,NaN,NaN,NaN,NaN,NaN,_,NaN,BC,NaN,...,M,NaN,NaN,NaN,NaN,5.0,NaN,NaN,2018-10-02T11:02:40.287+07,NaN
5,172,NaN,5.0,53.0,BUS,1234,_,Y,BC,PAF,...,F,Y,YPU,CWP,PA,2.0,2,22.0,2018-09-28T13:30:10.064+08,HYP
6,174,NaN,0.0,NaN,NOA,051517261,_,Y,NID,NaN,...,F,NaN,NaN,ASH,FL,2.0,2,NaN,2018-10-02T10:31:02.828+07,IHD MAL
7,175,NaN,0.0,NaN,OTH,180146096 (01),_,Y,NID,NaN,...,F,NaN,NaN,ASH,FL,5.0,>2,NaN,2018-10-02T10:53:38.679+07,IHD
8,177,NaN,0.0,NaN,OTH,180279323(01),_,N,NID,NaN,...,F,NaN,NaN,SPO,FL,5.0,NaN,NaN,2018-10-02T11:24:00.763+07,STR IHD
9,180,NaN,NaN,NaN,NaN,NaN,_,Y,BC,NaN,...,F,NaN,NaN,SPO,FL,5.0,1,NaN,2018-10-02T11:27:22.432+07,NaN


In [3]:
#resident_df.T[1:]
resident_df.T

,0,1,2,3,4,5,6,7,8,9,...,3949,3950,3951,3952,3953,3954,3955,3956,3957,3958
_id,173,176,178,181,179,172,174,175,177,180,...,4286,4287,4288,4289,4290,4291,4292,4293,4294,4295
_index_map,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
health_information.alcohol_within_week,NaN,NaN,NaN,NaN,NaN,5,0,0,0,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
health_information.times_exposed_to_t_smoke,NaN,NaN,NaN,NaN,NaN,53,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
personal_info.source_of_income,NaN,NaN,NaN,NaN,NaN,BUS,NOA,OTH,OTH,NaN,...,EMP,EMP,NaN,EMP,OTH,OTH,NaN,NaN,OTH,NaN
identity_consent.personal_id_no,NaN,NaN,l,NaN,NaN,1234,051517261,180146096 (01),180279323(01),NaN,...,NaN,NaN,NaN,180727256,NaN,NaN,NaN,NaN,NaN,NaN
identity_consent.town,_,_,_,_,_,_,_,_,_,_,...,_,_,_,_,_,_,_,_,_,_
dental_health.owning_toothbrush,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,N,Y,...,Y,Y,NaN,Y,Y,Y,NaN,NaN,N,N
identity_consent.identity_document,BC,BC,BC,BC,BC,BC,NID,NID,NID,BC,...,BC,BC,BC,NID,BC,BC,BC,BC,BC,BC
health_information.how_taking_prescribed_dose,NaN,NaN,NaN,NaN,NaN,PAF,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
new_residents_df = pd.DataFrame()
new_resident_df = resident_df.T.apply(concat_values)

new_resident_df

,0,1,2,3,4,5,6,7,8,9,...,3949,3950,3951,3952,3953,3954,3955,3956,3957,3958
_id,173,176,178,181,179,172,174,175,177,180,...,4286,4287,4288,4289,4290,4291,4292,4293,4294,4295
_index_map,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
health_information.alcohol_within_week,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:5.0,health_information.alcohol_within_week:0.0,health_information.alcohol_within_week:0.0,health_information.alcohol_within_week:0.0,health_information.alcohol_within_week:nan,...,health_information.alcohol_within_week:0.0,health_information.alcohol_within_week:0.0,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan
health_information.times_exposed_to_t_smoke,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:53.0,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,...,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan
personal_info.source_of_income,personal_info.source_of_income:nan,personal_info.source_of_income:nan,personal_info.source_of_income:nan,personal_info.source_of_income:nan,personal_info.source_of_income:nan,personal_info.source_of_income:BUS,personal_info.source_of_income:NOA,personal_info.source_of_income:OTH,personal_info.source_of_income:OTH,personal_info.source_of_income:nan,...,personal_info.source_of_income:EMP,personal_info.source_of_income:EMP,personal_info.source_of_income:nan,personal_info.source_of_income:EMP,personal_info.source_of_income:OTH,personal_info.source_of_income:OTH,personal_info.source_of_income:nan,personal_info.source_of_income:nan,personal_info.source_of_income:OTH,personal_info.source_of_income:nan
identity_consent.personal_id_no,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:l,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:1234,identity_consent.personal_id_no:051517261,identity_consent.personal_id_no:180146096 (01),identity_consent.personal_id_no:180279323(01),identity_consent.personal_id_no:nan,...,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:180727256,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan
identity_consent.town,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,...,identity_consent.town:_,identit

In [5]:
def concat_values(items):
    _list = []
    _index_map = ''
    _id = ''
    
    for index ,value in items.iteritems():
        _new_item = ''
        
    
        if index == "_id":
            _id = value
            #print(_index_map)
            _list.append(_id)
           
        elif index == "_index_map":
            _index_map = value
            
            _list.append(_index_map)
            #TODO: MUST BE A FUNCTION --> Parser of index_map
        else:
            if len(index) > 1 :
                
                _new_index = index.split(sep='.')
                
                
                if pd.isna(_index_map) :
                    _new_index[0] = _new_index[0]
                
                else:
                    
                    _new_index[0] = _new_index[0] + '|' + str(_index_map)
                    
                _edited_index = '.'.join(_new_index)
                _edited_index

                    #===========
                    #_new_item = index + ':"' + str(value) + '"'
                _new_item = _edited_index + ":" + str(value) + ""
                _list.append(_new_item)
            else:
                continue
            
    return _list    

In [7]:
new_resident_df = new_resident_df.drop(index='_index_map')
#TODO: Change idnetifier _id formid id accordingly
#new_resident_df = new_resident_df.drop(index='_id')
new_resident_df 

,0,1,2,3,4,5,6,7,8,9,...,3949,3950,3951,3952,3953,3954,3955,3956,3957,3958
_id,173,176,178,181,179,172,174,175,177,180,...,4286,4287,4288,4289,4290,4291,4292,4293,4294,4295
health_information.alcohol_within_week,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:5.0,health_information.alcohol_within_week:0.0,health_information.alcohol_within_week:0.0,health_information.alcohol_within_week:0.0,health_information.alcohol_within_week:nan,...,health_information.alcohol_within_week:0.0,health_information.alcohol_within_week:0.0,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan,health_information.alcohol_within_week:nan
health_information.times_exposed_to_t_smoke,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:53.0,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,...,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan,health_information.times_exposed_to_t_smoke:nan
personal_info.source_of_income,personal_info.source_of_income:nan,personal_info.source_of_income:nan,personal_info.source_of_income:nan,personal_info.source_of_income:nan,personal_info.source_of_income:nan,personal_info.source_of_income:BUS,personal_info.source_of_income:NOA,personal_info.source_of_income:OTH,personal_info.source_of_income:OTH,personal_info.source_of_income:nan,...,personal_info.source_of_income:EMP,personal_info.source_of_income:EMP,personal_info.source_of_income:nan,personal_info.source_of_income:EMP,personal_info.source_of_income:OTH,personal_info.source_of_income:OTH,personal_info.source_of_income:nan,personal_info.source_of_income:nan,personal_info.source_of_income:OTH,personal_info.source_of_income:nan
identity_consent.personal_id_no,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:l,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:1234,identity_consent.personal_id_no:051517261,identity_consent.personal_id_no:180146096 (01),identity_consent.personal_id_no:180279323(01),identity_consent.personal_id_no:nan,...,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:180727256,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan,identity_consent.personal_id_no:nan
identity_consent.town,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,...,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identity_consent.town:_,identi

In [10]:
test_foo_df = []
test_foo_df = new_resident_df.apply(insert_concat)


flat= list(itertools.chain.from_iterable(test_foo_df))
flat_df = pd.DataFrame.from_dict(flat)
flat_df

,id,value
0,173,health_information.alcohol_within_week:nan
1,173,health_information.times_exposed_to_t_smoke:nan
2,173,personal_info.source_of_income:nan
3,173,identity_consent.personal_id_no:nan
4,173,identity_consent.town:_
5,173,dental_health.owning_toothbrush:nan
6,173,identity_consent.identity_document:BC
7,173,health_information.how_taking_prescribed_dose:nan
8,173,pregnancy.place_receive_anc:nan
9,173,personal_info.first_name:ម៉ូលីតា


In [11]:
flat_df.to_csv('file/parsed_output/test.csv', encoding='utf-8', mode='w', index=False)

In [9]:
def insert_concat(items):
    test_df = pd.DataFrame()
    _test_obj = []
    
    for index ,value in items.iteritems():
        #print(value)
        _kv = {}
        
        #TODO: change _id here in formid, id or id
        if index == '_id':
        #if index == 'id':
            #print(index)
            _test_id = value
            continue
        else:
            _kv["id"] = _test_id
            _kv["value"] = value
            
        _test_obj.append(_kv)

    #print('========')
    #print(_test_obj)
    #print('-------')
    #return test_df
    return _test_obj
    

In [12]:
hc_df = pd.DataFrame()
hc_df = pd.read_csv( 'file/parsed_output/test.csv',header=None) 
#hc_df.reset_index(level=0, inplace=True)
hc_df

/Users/ghost/src/virtual_env/jupyther3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1
0,id,value
1,173,health_information.alcohol_within_week:nan
2,173,health_information.times_exposed_to_t_smoke:nan
3,173,personal_info.source_of_income:nan
4,173,identity_consent.personal_id_no:nan
5,173,identity_consent.town:_
6,173,dental_health.owning_toothbrush:nan
7,173,identity_consent.identity_document:BC
8,173,health_information.how_taking_prescribed_dose:nan
9,173,pregnancy.place_receive_anc:nan


In [17]:
import csv
import json
import sys
import os

def setValue(value, field, it):
    if isinstance(value, dict) or isinstance(value, list) and field not in it: # prevent override
            it[field] = value
    if not isinstance(value, dict) and not isinstance(value, list):
            it[field] = value

def findField(ds, k, keyVal, it):
    it = ds[k]
    if isinstance(it, list): # if array then find correct index
        index = int(keyVal[1])
        if len(it) > index:
            it = it[index]
    return it

def setAttribute(profile, lastKeys, field, value):
    it = {}
    for key in lastKeys:
        keyVal = key.split('|')
        k = keyVal[0]
        if not it:
            it = findField(profile, k, keyVal, it)
        else:
            it = findField(it, k, keyVal, it)

    if isinstance(it, list):
        index = int(lastKeys[-1].split('|')[1]) # get index from ['contactnumber/0']
        if len(it) > index: # find element and element attribute to set value
            setValue(value, field, it[index])
        else: # list is empty so create an element
            for i in range(len(it), index + 1, 1):
                it.append({})
            it[index][field] = value
    else:
        setValue(value, field, it)

def defineField(profile, field):
    keyVal = field.split('|')
    if len(keyVal) > 1:
        key = keyVal[0]
        if key not in profile:
            profile[key] = []
    elif field not in profile:
        profile[field] = {}

profiles = []

def properValue(value):
    proper = None
    try:
        proper = eval(value)
    except:
        proper = value
    return proper

# app starts here


print('parser running...')
with open('file/parsed_output/test.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    profile = {}
    lastId = None
    firstProfile = True
    for row in csv_reader: # read the csv line by line
        if line_count > 0: # don't read columns
            # print(row[1].rsplit(':', 1))
            fieldValueRaw = row[1]
            fieldValue = fieldValueRaw.split(":", 1)
            rawFields = [x.strip() for x in fieldValue[0].split('.')]
            rawFields[-1] = rawFields[-1] + ':' + fieldValue[1]
            # stop updating profile when id changes
            # add profile and build it afterwards
            if lastId != row[0]: # new id means add and build profile
                if not firstProfile:
                    profile = {} # reset profile and build it
                profile['id'] = row[0]
                profiles.append(profile) # this profile will be built below
                lastId = row[0]
                firstProfile = False

            lastKeys = []
            for field in rawFields: # build the profile up
                keyVal = field.split(':', 1) # field and value format, we separate it
                if len(keyVal) == 1: # ['answers'] if field doesn't contain value we build it
                    if len(lastKeys) == 0:
                        defineField(profile, field)
                    else:
                        _keyVal = field.split('|')
                        if len(_keyVal) > 1:
                            setAttribute(profile, lastKeys, _keyVal[0], [])
                        else:
                            setAttribute(profile, lastKeys, field, {})
                    lastKeys.append(field)
                else: # ['type', 'mobile']
                    if len(lastKeys) == 0:
                        profile[keyVal[0]] = properValue(keyVal[1])
                    else:
                        setAttribute(profile, lastKeys, keyVal[0], properValue(keyVal[1]))
                    lastKeys.append(keyVal[0])
        line_count += 1

#os.makedirs('output', exist_ok=True)
file = open('file/parsed_output/test.json', 'w')
file.write(json.dumps(profiles, indent=4, sort_keys=True))
#file.write(json.dumps(profiles))
file.close()
print('parser completed...')

parser running...
parser completed...
